# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loading the cities csv file created in WeatherPy to a dataframe
city_vacation_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
city_vacation_df = city_vacation_df[['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']]
city_vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Puerto Ayora,-0.74,-90.35,73.54,88,66,10.00,EC,1624152237
1,Pochutla,15.74,-96.47,81.82,90,99,3.60,MX,1624152237
2,San Patricio,28.02,-97.52,85.95,67,67,12.44,US,1624152237
3,Cabo San Lucas,22.89,-109.91,96.80,72,5,17.27,MX,1624152238
4,Yaan,7.38,8.57,74.46,92,100,6.31,NG,1624152238


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps using the api key
gmaps.configure(api_key=g_key)

In [4]:
# Using the latitude and longitude columns as locations and copying to a new dataframe
# Using the humidity column as weight and copying to a new dataframe
locations = city_vacation_df[['Lat', 'Lng']]
humidity = city_vacation_df['Humidity']

In [5]:
# Creating the Heatmap using gmaps for the humidity of the cities
fig = gmaps.figure(zoom_level=2, center=(0,0))

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=300, point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Creating a new dataframe with max temperature greater than 70 Farenheit and less than 80 Farenheit and 
# with wind speed less than 10 and
# cloudiness equal to 0
# dropping null vales
# Adding a new column Hotel Name
hotel_df = city_vacation_df.loc[(city_vacation_df['Max Temp'] < 80) & (city_vacation_df['Max Temp'] > 70)
                                               & (city_vacation_df['Wind Speed'] < 10) 
                                               & (city_vacation_df['Cloudiness'] == 0)].dropna()
hotel_df['Hotel Name'] = ' '
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
86,Changji,44.02,87.32,79.59,34,0,8.95,CN,1624152263,
112,Nikolskoye,59.70,30.79,71.49,77,0,5.28,RU,1624152270,
124,Ţahţā,26.77,31.50,79.50,25,0,4.38,EG,1624152273,
300,Kohlu,29.90,69.25,79.27,44,0,2.68,PK,1624152326,
303,Fukuechō,32.69,128.84,79.16,61,0,8.05,JP,1624152265,
333,Buraidah,26.33,43.98,79.25,13,0,4.43,SA,1624152338,
386,Āsmār,35.03,71.36,75.85,32,0,7.74,AF,1624152353,
403,Mizdah,31.45,12.98,76.87,28,0,6.22,LY,1624152360,
441,Korla,41.76,86.15,76.08,26,0,6.40,CN,1624152372,
542,Tricase,39.93,18.36,71.82,63,0,8.48,IT,1624152404,


In [7]:
# Setting the parameters for the google place api search to find hotels within 5000m
# Adding the Hotel name from the json response to the hotel_df dataframe and printing any cities for which no cities were 
# found within 5000m
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from hotel df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_name["results"][0]["name"]
    except:
        print('No hotel found within 5000m in: ' + row['City'])
    
 

No hotel found within 5000m in: Āsmār


In [8]:
# Displaying the hotel_df
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
86,Changji,44.02,87.32,79.59,34,0,8.95,CN,1624152263,Lvye Holiday Resort
112,Nikolskoye,59.70,30.79,71.49,77,0,5.28,RU,1624152270,Tourist House - Sablino
124,Ţahţā,26.77,31.50,79.50,25,0,4.38,EG,1624152273,ماستر تك
300,Kohlu,29.90,69.25,79.27,44,0,2.68,PK,1624152326,Captain Shaheed Mir Jahangir Abad kohlu
303,Fukuechō,32.69,128.84,79.16,61,0,8.05,JP,1624152265,Goto Guesthouse Utojuku & SA Car Rental
333,Buraidah,26.33,43.98,79.25,13,0,4.43,SA,1624152338,فندق راحة السلام
386,Āsmār,35.03,71.36,75.85,32,0,7.74,AF,1624152353,
403,Mizdah,31.45,12.98,76.87,28,0,6.22,LY,1624152360,فندق بنيه تحتيه
441,Korla,41.76,86.15,76.08,26,0,6.40,CN,1624152372,Gulisitan Hotel
542,Tricase,39.93,18.36,71.82,63,0,8.48,IT,1624152404,Hotel Adriatico


In [13]:
# Removing cities for which no hotels were found within 5000m
hotel_df = hotel_df.loc[hotel_df['Hotel Name']!=' ']
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
86,Changji,44.02,87.32,79.59,34,0,8.95,CN,1624152263,Lvye Holiday Resort
112,Nikolskoye,59.70,30.79,71.49,77,0,5.28,RU,1624152270,Tourist House - Sablino
124,Ţahţā,26.77,31.50,79.50,25,0,4.38,EG,1624152273,ماستر تك
300,Kohlu,29.90,69.25,79.27,44,0,2.68,PK,1624152326,Captain Shaheed Mir Jahangir Abad kohlu
303,Fukuechō,32.69,128.84,79.16,61,0,8.05,JP,1624152265,Goto Guesthouse Utojuku & SA Car Rental
333,Buraidah,26.33,43.98,79.25,13,0,4.43,SA,1624152338,فندق راحة السلام
403,Mizdah,31.45,12.98,76.87,28,0,6.22,LY,1624152360,فندق بنيه تحتيه
441,Korla,41.76,86.15,76.08,26,0,6.40,CN,1624152372,Gulisitan Hotel
542,Tricase,39.93,18.36,71.82,63,0,8.48,IT,1624152404,Hotel Adriatico


In [14]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
display_city = list(hotel_df['City'])

In [15]:
# Adding marker layer on top of heat map

hotel_layer = gmaps.marker_layer(locations,
    hover_text= display_city,                            
    info_box_content=hotel_info
)


fig = gmaps.figure(zoom_level=2, center=(0,0))

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))